# Food Image Classifcation

### Dataset

We are going learn [food images](https://www.kaggle.com/kmader/food41) from kaggle

### Use the kaggle-cli to download the image

```kaggle datasets download -d kmader/food41```

In [1]:
import os
import torch

In [2]:
CUDA  = torch.cuda.is_available()

In [3]:
HOME = os.environ["HOME"]
HOME

'/home/paperspace'

In [4]:
DATA  = HOME+"/.kaggle/datasets/kmader/food41/"
DATA

'/home/paperspace/.kaggle/datasets/kmader/food41/'

In [20]:
# !!du -sh {DATA}*

['5.3G\t/home/paperspace/.kaggle/datasets/kmader/food41/food41.zip',
 '343M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n1000_r384x384x3.h5',
 '18M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r32x32x1.h5',
 '30M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r32x32x3.h5',
 '68M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r64x64x1.h5',
 '115M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r64x64x3.h5',
 '25M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r128x128x1.h5',
 '44M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r128x128x3.h5',
 '1.9M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r32x32x1.h5',
 '3.2M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r32x32x3.h5',
 '6.7M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r64x64x1.h5',
 '12M\t/home/paperspace/.kaggle/datasets/

In [23]:
# %mkdir -p {DATA}images
# %mkdir -p {DATA}meta
# %ls -l {DATA}

total 11240352
-rw-rw-r-- 1 paperspace paperspace 5681757245 Aug  2 01:45 food41.zip
-rw-rw-r-- 1 paperspace paperspace  359407496 Aug  2 01:45 food_c101_n1000_r384x384x3.h5
-rw-rw-r-- 1 paperspace paperspace   18236331 Aug  2 01:45 food_c101_n10099_r32x32x1.h5
-rw-rw-r-- 1 paperspace paperspace   30452874 Aug  2 01:45 food_c101_n10099_r32x32x3.h5
-rw-rw-r-- 1 paperspace paperspace   70546750 Aug  2 01:45 food_c101_n10099_r64x64x1.h5
-rw-rw-r-- 1 paperspace paperspace  120306214 Aug  2 01:45 food_c101_n10099_r64x64x3.h5
-rw-rw-r-- 1 paperspace paperspace   26042428 Aug  2 01:45 food_test_c101_n1000_r128x128x1.h5
-rw-rw-r-- 1 paperspace paperspace   45897229 Aug  2 01:45 food_test_c101_n1000_r128x128x3.h5
-rw-rw-r-- 1 paperspace paperspace    1938115 Aug  2 01:45 food_test_c101_n1000_r32x32x1.h5
-rw-rw-r-- 1 paperspace paperspace    3294763 Aug  2 01:45 food_test_c101_n1000_r32x32x3.h5
-rw-rw-r-- 1 paperspace paperspace    6968764 Aug  2 01:45 food_test_c101_n1000_r64x64x1.h5
-rw-rw-r--

Unzip and calculate the line of sub-folders: categories

In [24]:
# !!cd {DATA}/images; unzip ../images.zip > unzip.log;rm -rf unzip.log; ls -l|wc -l

['102']

In [29]:
# !!cd {DATA}/meta; unzip -q ../meta.zip; ls -l meta

['total 4140',
 '-rw-rw-r-- 1 paperspace paperspace    1184 Jul  9  2014 classes.txt',
 '-rw-rw-r-- 1 paperspace paperspace    1184 Sep 23  2013 labels.txt',
 '-rw-rw-r-- 1 paperspace paperspace  566868 Sep 23  2013 test.json',
 '-rw-rw-r-- 1 paperspace paperspace  489429 Sep 23  2013 test.txt',
 '-rw-rw-r-- 1 paperspace paperspace 1697751 Sep 21  2013 train.json',
 '-rw-rw-r-- 1 paperspace paperspace 1468812 Sep 21  2013 train.txt']

In [5]:
META = DATA + "meta/meta/"
META

'/home/paperspace/.kaggle/datasets/kmader/food41/meta/meta/'

In [6]:
for m_file in os.listdir(META):
    print(m_file, "preview","="*20)
    print(open(META+m_file).read()[:50])

train.json preview ====================
{"churros": ["churros/1004234", "churros/1013460",
classes.txt preview ====================
apple_pie
baby_back_ribs
baklava
beef_carpaccio
be
train.txt preview ====================
apple_pie/1005649
apple_pie/1014775
apple_pie/1026
test.json preview ====================
{"churros": ["churros/1061830", "churros/1064042",
test.txt preview ====================
apple_pie/1011328
apple_pie/101251
apple_pie/10343
labels.txt preview ====================
Apple pie
Baby back ribs
Baklava
Beef carpaccio
Be


### Datasets

In [7]:
IMG = DATA+"images/"
IMG

'/home/paperspace/.kaggle/datasets/kmader/food41/images/'

In [8]:
from torch.utils.data import DataLoader
from torch import nn
from ray.matchbox import Trainer
from torchvision.models.densenet import densenet121 as feature_extractor
from torch.nn import functional as F
from torch.optim import Adam
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

In [9]:
SCALE = 224
transform = transforms.Compose([
    transforms.Resize((SCALE,SCALE)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
])

In [10]:
img_set = ImageFolder(IMG,transform = transform, )

#### Train /Valid Split

In [11]:
import numpy as np

In [12]:
trn_set = ImageFolder(IMG,transform = transform, )
val_set = ImageFolder(IMG,transform = transform, )

In [13]:
val_pick = np.random.rand(len(img_set.samples))>0.95
trn_pick = ~val_pick

In [14]:
trn_set.samples = np.array(img_set.samples)[trn_pick].tolist()
val_set.samples = np.array(img_set.samples)[val_pick].tolist()

In [15]:
trn_set.imgs = trn_set.samples
val_set.imgs = val_set.samples

In [16]:
len(trn_set),len(val_set)

(96030, 4970)

#### Test dataset

In [18]:
gen = iter(DataLoader(trn_set,batch_size=2,shuffle=True))
next(gen)

[tensor([[[[-0.1314, -0.0458,  0.0227,  ...,  0.0056,  0.0398, -0.0629],
           [-0.1143, -0.0458, -0.1314,  ...,  0.0056, -0.0972, -0.1486],
           [-0.1143, -0.1657, -0.1314,  ..., -0.0287, -0.0972, -0.0972],
           ...,
           [-0.4739, -0.5082,  0.0056,  ..., -1.0048, -1.0562, -0.9534],
           [-0.6109, -0.6109, -0.5253,  ..., -0.9363, -0.9534, -0.9192],
           [-0.6623, -0.6965, -0.6452,  ..., -0.9020, -0.8849, -0.8678]],
 
          [[-1.0903, -1.0553, -1.0028,  ..., -1.0553, -1.0378, -1.1429],
           [-1.0728, -1.0378, -1.1429,  ..., -1.0553, -1.1604, -1.1779],
           [-1.0378, -1.1779, -1.1253,  ..., -1.0728, -1.1253, -1.0903],
           ...,
           [-1.4230, -1.4580, -0.8803,  ..., -1.8782, -1.8957, -1.7731],
           [-1.4930, -1.5280, -1.4580,  ..., -1.8606, -1.8782, -1.8431],
           [-1.5105, -1.5980, -1.5980,  ..., -1.8431, -1.8256, -1.8256]],
 
          [[-1.2293, -1.1770, -1.1421,  ..., -1.3339, -1.3164, -1.4384],
           [-

In [19]:
class Flatten(nn.Module):
    def forward(self, input):
        """
        a pytorch version of Flatten layer
        """
        return input.view(input.size(0), -1)

In [20]:
def argmax(x):
    """
    Arg max of a torch tensor (2 dimensional, dim=1)
    :param x:  torch tensor
    :return: index the of the max
    """
    return torch.max(x, dim=1)[1]

def accuracy(y_pred, y_true):
    """

    :param y_pred: predition of y (will be argmaxed)
    :param y_true: true label of y (index)
    :return:
    """
    return (argmax(y_pred) == y_true).float().mean()

In [21]:
def save_model(model,path):
    """
    model:pytorch model
    path:save to path, end with pkl
    """
    torch.save(model.state_dict(), path)
    
def load_model(model,path):
    model.load_state_dict(torch.load(path))

## Transfer Learning

In [22]:
conv_model = feature_extractor(pretrained=True)

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [23]:
conv_layers = conv_model.features

In [25]:
FEATURE_WIDTH = 1024

In [26]:
class  top_half(nn.Module):
    def __init__(self,ks = 7):
        super(top_half,self).__init__()
        self.ks = ks
        self.classifier = nn.Linear(FEATURE_WIDTH,len(img_set.classes),bias = True)
        self.flatten = Flatten()
        nn.init.constant_(self.classifier.weight, 1)
        nn.init.constant_(self.classifier.bias, 0)
        
    def forward(self,x):
        x = F.relu(x,inplace=True)
        x = F.avg_pool2d(x,kernel_size = self.ks, stride = 1 )
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [27]:
top_half_  = top_half()

if CUDA:
    top_half_.cuda()
    conv_layers.cuda()

In [28]:
def action(*args,**kwargs):
    """
    single training step, 
    take in data, spit out loss/ metric
    and 
    """
    x,y = args[0]
    y = torch.LongTensor(np.array(y).astype(int))
    if CUDA:
        x,y = x.cuda(),y.cuda()
    opt.zero_grad()
    y_ = top_half_(conv_layers(x))
    
    loss = loss_func(y_,y)
    acc = accuracy(y_,y)
    
    loss.backward()
    opt.step()
    
    return {"loss":loss.item(),
            "acc":acc.item()}

def val_action(*args,**kwargs):
    x,y = args[0]
    y = torch.LongTensor(np.array(y).astype(int))
    
    if CUDA:
        x,y = x.cuda(),y.cuda()
    y_ = top_half_(conv_layers(x))
    
    loss = loss_func(y_,y)
    acc = accuracy(y_,y)
    
    return {"loss":loss.item(),
            "acc":acc.item()}

In [29]:
loss_func = nn.CrossEntropyLoss()
opt = Adam(top_half_.parameters())

trainer = Trainer(trn_set, val_dataset = val_set, batch_size = 32, print_on = 5)

trainer.action = action
trainer.val_action = val_action

In [30]:
# load_model(dense_conv2,"food_dense_conv2.0.0.1.npy")
# load_model(top_half_,"food_top.0.0.1.npy")

In [ ]:
trainer.train(1)

😎[val_ep_0_i_33]	acc	0.548😂	loss	1.832:  22%|██▏       | 34/156 [00:21<01:16,  1.60it/s]

In [31]:
save_model(conv_layers,"convlayers2.0.0.1.npy")
save_model(top_half_,"food_top.0.0.1.npy")

### Excercise

Please work on at least 2 of the following challenge

1. Optimize all the layers instead of only linear classifier
2. Optimize the last convblock(the conv block close the linear layer) and linear classifier
3. Try other image classify datasets, like [monekey image set](https://www.kaggle.com/slothkong/10-monkey-species) or [flower classifying problem](https://www.kaggle.com/alxmamaev/flowers-recognition) or [blood cell images](https://www.kaggle.com/paultimothymooney/blood-cells). You'll pretty soon find out convolutional neural network is a universal tool for this kind of problem
4. Try keras to work out a better accuaracy, keras import pretrained model by ```from keras.application import ... ```

how we break down a pytorch model to several pytorch models:

```python
dense_conv1 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["conv0","norm0","relu0","pool0","denseblock1","transition1",
                                                                                   "denseblock2","transition2","denseblock3","transition3",]])

dense_conv2 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["denseblock4","norm5"]])
```

